In [1]:
import numpy as np
from tabulate import tabulate
import random as rd
from threading import Thread
from time import sleep

### Definição da classe e seus métodos

In [2]:
class Banker:
    def __init__(self, process_id, available, allocation, maximum):
        self.process_id = process_id.copy()
        self.available = available.copy()
        self.allocation = allocation.copy()
        self.maximum = maximum.copy()
        self.need = self.maximum - self.allocation
        self.execution_order = []
        self.last_process = self.process_id.max()
   
    def insert_process(self, request):

        self.maximum = np.append(self.maximum, request, axis=0)
        self.allocation = np.append(self.allocation, np.zeros_like(request), axis=0)
        self.need = np.append(self.need, request, axis=0)
        self.process_id = np.append(self.process_id, [self.last_process+1], axis=0)
        self.last_process += 1

    def execute_process(self, idx):
        # Deleta o processo das listas e torna disponivel seus recursos
        self.available += self.allocation[idx]
        self.allocation = np.delete(self.allocation, idx, axis=0)
        self.maximum = np.delete(self.maximum, idx, axis=0)
        self.need = self.maximum - self.allocation
        self.process_id = np.delete(self.process_id, idx, axis=0)

    def is_safe(self):
        print(f"{'-'*20} Estado inicial {'-'*20}")
        print(self)        

        while(True):   
            # Quais processos são passíveis de ser finalizados         
            self.possible = np.all(self.need <= self.available, axis=1)
            
            # Caso não haja mais processos
            if(self.possible.size == 0):
                print("Sem mais processos para computar")
                break

            # Caso não haja mais nenhum processo possível então há um estado unsafe
            if not any(self.possible):
                print("Sem mais processos possíveis de serem executados, estado unsafe")
                return False, []

            # Caso haja processo possível execute-o
            print(f"O(s) processo(s) {*self.process_id[self.possible],} pode(m) ser executado(s)")
            idx = np.argmax(self.possible)
            process_id = self.process_id[idx]
            self.execute_process(idx)   
            print(f"Processo {process_id} executado")

            # Grave a orde de execução
            self.execution_order.append(process_id)
            print(f"\n{'-'*20} Estado atual {'-'*20}")
            print(self)

        return True, self.execution_order
    
    def __str__(self):
        headers = [f"R{i}" for i in range(1, self.available.size+1)]
        print("Maximum")
        print(tabulate(zip(self.process_id, *self.maximum.T), headers=["PID"] + headers, tablefmt="fancy_grid"))

        print("Allocation")
        print(tabulate(zip(self.process_id, *self.allocation.T), headers=["PID"] + headers, tablefmt="fancy_grid"))

        print("Need")
        print(tabulate(zip(self.process_id, *self.need.T), headers=["PID"] + headers, tablefmt="fancy_grid"))

        print("Available")
        print(tabulate([self.available], headers=headers, tablefmt="fancy_grid"))

        return ""

    def __repr__(self):
        return self.__str__()

### Caso pré-definido

In [9]:
n_rec = 6
n_processes = 5

# Testes gerados de modo aleatório
available = np.array(rd.sample(range(0,10), n_rec))
allocation = np.random.randint(low=0, high=11, size=(n_processes, n_rec))
maximum = allocation + np.random.randint(low=0, high=5, size=(n_processes, n_rec)) 
process_id = np.arange(1, n_processes+1)

banker = Banker(process_id, available, allocation, maximum)
banker.is_safe()

-------------------- Estado inicial --------------------
Maximum
╒═══════╤══════╤══════╤══════╤══════╤══════╤══════╕
│   PID │   R1 │   R2 │   R3 │   R4 │   R5 │   R6 │
╞═══════╪══════╪══════╪══════╪══════╪══════╪══════╡
│     1 │    6 │    7 │    5 │    8 │    6 │    9 │
├───────┼──────┼──────┼──────┼──────┼──────┼──────┤
│     2 │    4 │    9 │   14 │    1 │    7 │   10 │
├───────┼──────┼──────┼──────┼──────┼──────┼──────┤
│     3 │    9 │    6 │    6 │   10 │    4 │   13 │
├───────┼──────┼──────┼──────┼──────┼──────┼──────┤
│     4 │   14 │    9 │   11 │    3 │   12 │    9 │
├───────┼──────┼──────┼──────┼──────┼──────┼──────┤
│     5 │    1 │    6 │   10 │   11 │    6 │    5 │
╘═══════╧══════╧══════╧══════╧══════╧══════╧══════╛
Allocation
╒═══════╤══════╤══════╤══════╤══════╤══════╤══════╕
│   PID │   R1 │   R2 │   R3 │   R4 │   R5 │   R6 │
╞═══════╪══════╪══════╪══════╪══════╪══════╪══════╡
│     1 │    2 │    7 │    1 │    7 │    3 │    9 │
├───────┼──────┼──────┼──────┼──────┼───

(True, [1, 2, 3, 4, 5])

### Tentativa de inserção automática

In [6]:
n_new_processes=10
if __name__ == '__main__':
    while(n_new_processes>0):
        new_process = np.random.randint(low=0, high=10, size=(1, n_rec))
        Thread(target = banker.insert_process, kwargs={"request": new_process}).start()
        Thread(target = banker.is_safe).start()
        n_new_processes -= 1
        sleep(15)

# print(f"Os processos foram executados na seguinte orderm {*banker.execution_order,}")

-------------------- Estado inicial --------------------
Maximum
╒═══════╤══════╤══════╤══════╤══════╤══════╤══════╕
│   PID │   R1 │   R2 │   R3 │   R4 │   R5 │   R6 │
╞═══════╪══════╪══════╪══════╪══════╪══════╪══════╡
│     6 │    4 │    0 │    2 │    5 │    6 │    1 │
╘═══════╧══════╧══════╧══════╧══════╧══════╧══════╛
Allocation
╒═══════╤══════╤══════╤══════╤══════╤══════╤══════╕
│   PID │   R1 │   R2 │   R3 │   R4 │   R5 │   R6 │
╞═══════╪══════╪══════╪══════╪══════╪══════╪══════╡
│     6 │    0 │    0 │    0 │    0 │    0 │    0 │
╘═══════╧══════╧══════╧══════╧══════╧══════╧══════╛
Need
╒═══════╤══════╤══════╤══════╤══════╤══════╤══════╕
│   PID │   R1 │   R2 │   R3 │   R4 │   R5 │   R6 │
╞═══════╪══════╪══════╪══════╪══════╪══════╪══════╡
│     6 │    4 │    0 │    2 │    5 │    6 │    1 │
╘═══════╧══════╧══════╧══════╧══════╧══════╧══════╛
Available
╒══════╤══════╤══════╤══════╤══════╤══════╕
│   R1 │   R2 │   R3 │   R4 │   R5 │   R6 │
╞══════╪══════╪══════╪══════╪══════╪═════